In [1]:
import tensorflow as tf
from tensorflow import keras

from tf_quantization.layers.quant_conv2D_batch_layer import QuantConv2DBatchLayer
from tf_quantization.layers.quant_depthwise_conv2d_bn_layer import QuantDepthwiseConv2DBatchNormalizationLayer
from keras.layers import Conv2D, BatchNormalization, ReLU, DepthwiseConv2D

import tensorflow_datasets as tfds

In [2]:
def preprocess_image(data):
    """This method preprocess images to input format of Mobilenet"""
    data['image'] = (tf.image.resize(data['image'], (28, 28)) / 255.0)
    return data

In [3]:
# Load dataset
tr_ds = tfds.load("mnist", split="train")
tr_ds = tr_ds.map(preprocess_image)

train_ds = tr_ds \
    .map(lambda data: (data['image'], data['label'])) \
    .batch(64)

val_ds = tfds.load("mnist", split="test")
val_ds = tr_ds.map(preprocess_image)

validation_ds = val_ds \
    .map(lambda data: (data['image'], data['label'])) \
    .batch(64)

Metal device set to: Apple M2 Pro
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-02 16:33:23.639707: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-02 16:33:23.639748: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
# Create normal model with BatchNormalization
model1 = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    DepthwiseConv2D(kernel_size=(3, 3)),
    BatchNormalization(),
    ReLU(),
    Conv2D(filters=32, kernel_size=(3, 3)),
    BatchNormalization(),
    ReLU(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=100, activation="relu"),
    keras.layers.Dense(units=10, activation="softmax")
])

In [5]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 26, 26, 1)        10        
 Conv2D)                                                         
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 1)        4         
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 26, 26, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 32)        320       
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 32)       1

In [6]:
model1.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.01),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])

In [7]:
model1.fit(train_ds, epochs=1, validation_data=validation_ds)

2023-03-02 16:33:28.388320: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  1/938 [..............................] - ETA: 7:30 - loss: 2.5891 - accuracy: 0.0938

2023-03-02 16:33:28.687015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - ETA: 0s - loss: 0.4014 - accuracy: 0.9216

2023-03-02 16:33:37.902249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 14s 14ms/step - loss: 0.4014 - accuracy: 0.9216 - val_loss: 2.4464 - val_accuracy: 0.1253


In [8]:
model1.evaluate(validation_ds)

938/938 [==============================] - 5s 5ms/step - loss: 2.4464 - accuracy: 0.1253


[2.446354866027832, 0.12531666457653046]

In [9]:
# Try to create sequential model
model2 = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    QuantDepthwiseConv2DBatchNormalizationLayer(DepthwiseConv2D(kernel_size=(3, 3)), BatchNormalization(),
                                                quantize=False),
    ReLU(),
    QuantConv2DBatchLayer(Conv2D(filters=32, kernel_size=(3, 3)), BatchNormalization(), quantize=False),
    ReLU(),
    keras.layers.Flatten(),
    keras.layers.Dense(units=100, activation="relu"),
    keras.layers.Dense(units=10, activation="softmax")
])

In [10]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 depthwise_conv2d_1 (QuantDe  (None, 26, 26, 1)        14        
 pthwiseConv2DBatchNormaliza                                     
 tionLayer)                                                      
                                                                 
 re_lu_2 (ReLU)              (None, 26, 26, 1)         0         
                                                                 
 quant_conv2d_batch_layer (Q  (None, 24, 24, 32)       448       
 uantConv2DBatchLayer)                                           
                                                                 
 re_lu_3 (ReLU)              (None, 24, 24, 32)        0         
                                                      

In [14]:
import numpy as np

In [15]:
# Copy weights from original model to new one with folded batch normalization
w1 = model1.layers[1].get_weights()
w2 = model1.layers[2].get_weights()
for i in w2:
    w1.append(i)
weights1 = w1

# Copy weights from original model to new one with folded batch normalization
w4 = model1.layers[4].get_weights()
w5 = model1.layers[5].get_weights()
for i in w5:
    w4.append(i)
weights2 = w4
(weights1, weights2)

([array([[[[-0.18681449]],
  
          [[ 0.11694831]],
  
          [[ 0.39855558]]],
  
  
         [[[-0.57451683]],
  
          [[-0.28075325]],
  
          [[ 0.04749025]]],
  
  
         [[[ 0.10383015]],
  
          [[-0.05206235]],
  
          [[ 0.12018496]]]], dtype=float32),
  array([-0.05995787], dtype=float32),
  array([0.80708593], dtype=float32),
  array([-0.03637139], dtype=float32),
  array([-0.09828416], dtype=float32),
  array([0.04596099], dtype=float32)],
 [array([[[[-2.06751734e-01,  8.77170637e-03,  2.59070881e-02,
            -1.85876321e-02, -8.53303671e-02,  1.66377034e-02,
            -1.84857801e-01, -5.31768054e-02, -1.70761526e-01,
             1.16809935e-03, -3.12983729e-02,  5.03551513e-02,
            -6.12413883e-02, -2.43707988e-02, -2.91338004e-02,
            -4.74024490e-02, -3.85264377e-03, -3.87388915e-02,
             5.22219436e-03, -1.58642814e-01,  1.68463327e-02,
            -6.65784851e-02,  1.63492784e-02,  3.63156497e-02,
         

In [16]:
model2.layers[1].set_weights(weights1)
model2.layers[3].set_weights(weights2)

model2.layers[6].set_weights(model1.layers[8].get_weights())
model2.layers[7].set_weights(model1.layers[9].get_weights())

In [17]:
model2.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.01),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])

In [18]:
model2.evaluate(validation_ds)

 17/938 [..............................] - ETA: 6s - loss: 2.4439 - accuracy: 0.1222

2023-03-02 16:34:56.580256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 5s 5ms/step - loss: 2.4464 - accuracy: 0.1253


[2.4463553428649902, 0.12531666457653046]

In [16]:
model1.get_weights()

[array([[[[-0.00772099]],
 
         [[-0.4287056 ]],
 
         [[ 0.16604887]]],
 
 
        [[[ 0.19479099]],
 
         [[ 0.5932404 ]],
 
         [[-0.1316697 ]]],
 
 
        [[[-0.03558662]],
 
         [[ 0.44135955]],
 
         [[-0.28623256]]]], dtype=float32),
 array([0.01636761], dtype=float32),
 array([0.6940473], dtype=float32),
 array([-0.02690872], dtype=float32),
 array([0.09371328], dtype=float32),
 array([0.04709698], dtype=float32),
 array([[[[-4.43961293e-01,  2.63375137e-02,  1.27002932e-02,
           -2.06288751e-02,  4.06738035e-02, -2.46475767e-02,
           -1.78001717e-01, -5.26210904e-01,  1.71494395e-01,
           -5.35865268e-03,  2.25537106e-01, -2.92532891e-01,
           -4.11953665e-02, -2.82145083e-01,  4.17746454e-02,
           -4.52229977e-02,  9.64241400e-02, -1.03676692e-01,
           -6.03018045e-01, -1.69108778e-01, -4.26875144e-01,
            7.28268474e-02, -3.91288884e-02, -2.79690593e-01,
            2.48906836e-05, -5.51006906e-02, 

In [17]:
model2.get_weights()

[array([[[[-0.00772099]],
 
         [[-0.4287056 ]],
 
         [[ 0.16604887]]],
 
 
        [[[ 0.19479099]],
 
         [[ 0.5932404 ]],
 
         [[-0.1316697 ]]],
 
 
        [[[-0.03558662]],
 
         [[ 0.44135955]],
 
         [[-0.28623256]]]], dtype=float32),
 array([0.01636761], dtype=float32),
 array([0.6940473], dtype=float32),
 array([-0.02690872], dtype=float32),
 array([0.09371328], dtype=float32),
 array([0.04709698], dtype=float32),
 array([[[[-4.43961293e-01,  2.63375137e-02,  1.27002932e-02,
           -2.06288751e-02,  4.06738035e-02, -2.46475767e-02,
           -1.78001717e-01, -5.26210904e-01,  1.71494395e-01,
           -5.35865268e-03,  2.25537106e-01, -2.92532891e-01,
           -4.11953665e-02, -2.82145083e-01,  4.17746454e-02,
           -4.52229977e-02,  9.64241400e-02, -1.03676692e-01,
           -6.03018045e-01, -1.69108778e-01, -4.26875144e-01,
            7.28268474e-02, -3.91288884e-02, -2.79690593e-01,
            2.48906836e-05, -5.51006906e-02, 